# Monitoreo de Espacios Verdes   / 3 

### Idea, Secuencia y Suposiciones/Hipotesis

En este momento buscamos calcular ciertos estadisticos de las bandas de interes para cada roi de interes.
.
#### Suposiciones/Hipotesis

- Utilizamos solo la mascara de nubes provista por sentinel.

#### Idea

La idea consiste en caracterizar via algun estadistico el conjunto de puntos abarcado por cada parche para cada fecha disponible. El tamaño de cada parche es arbitrario asi como los estadisticos elegidos.
Esto nos va a permitir construir una serie temporal para un dado parche.

#### Secuencia

- Se procesa cada imagen y se genera un recorte sobre la ciudad en cuestion (parche)

- El parche obtenido se genera con cierto numero de bandas y se guarda.
    
- Las bandas que se utilizan se indica en el procesamiento y siempre corresponden con una resolucion de 10 m. En aquellos casos que la banda tenga menor resolucion se resamplea a 10 m.

- Una vez procesadas todas las imagenes satelitales (con la consecuente generacion de parches).Pasamos a la etapa de calculos de estadisticos sobre cada parche


In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import geopandas as gpd
import os
import glob
import matplotlib.pyplot as plt
from rasterio.plot import plotting_extent
from earthpy import plot
from pathlib import Path
import rasterio as rio
import json
import numpy as np
import pandas as pd

#
from eo_forge.io.SentinelLoaders import Sentinel2Loader

#
import sys

sys.path.append("..")

from src.ioCalc import (
    get_zonal_stats_indexes_all_geom,
    apply_mask_return_array,
    get_raster_stats,
    get_stats,
)

#
import dask
from dask.distributed import Client, progress
from dask.delayed import delayed

Los datos concernientes a Córdoba capital los extraemos del set general de datos

In [2]:
client = Client(n_workers=8, threads_per_worker=1,processes=True)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 62.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38123,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 62.65 GiB
Comm: tcp://127.0.0.1:38975,Total threads: 1
Dashboard: http://127.0.0.1:42133/status,Memory: 7.83 GiB
Nanny: tcp://127.0.0.1:43711,


In [3]:
gpd_cba = gpd.read_file("../data/cba/espacios-verdes-cba.gpkg")
gpd_cba_crs = gpd_cba.to_crs(32720)

In [4]:
gpd_cba.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 757 entries, 0 to 756
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   osm_id      757 non-null    object  
 1   fclass      757 non-null    object  
 2   name        191 non-null    object  
 3   cluster_id  757 non-null    float64 
 4   area_m2     757 non-null    float64 
 5   geometry    757 non-null    geometry
dtypes: float64(2), geometry(1), object(3)
memory usage: 35.6+ KB


La idea en lo que sigue es obtener la imagen satelital correspondiente a la zona y generar los parametros correspondientes (sin demasiada complejidad).

In [5]:
BASE_DIR = Path("../TMP-products-int")
os.makedirs(BASE_DIR, exist_ok=True)
# make crop dir
CROP_DIR = BASE_DIR / "images-croped"
os.makedirs(CROP_DIR, exist_ok=True)
#
IMAGES_RAW = "/data/sat-images/sentinel2"

In [6]:
f"Tenemos {len(os.listdir(IMAGES_RAW))} imagenes"

'Tenemos 276 imagenes'

In [8]:
crop_images = True
CROPPED_FILES_JSON='../data/cba/cropped-files.json'
CROPPED_FILES_CSV='../data/cba/cropped-files.csv'
if crop_images:
    #
    # Init class
    gpd_box = gpd.read_file("../data/cba/cordoba.geojson")
    BANDS = ["B02", "B03", "B04", "B05", "B06", "B07", "B8A", "B08", "B11", "B12"]
    sl = Sentinel2Loader(IMAGES_RAW, bands=BANDS, resolution=10, bbox=gpd_box)
    #
    delayed_crop = delayed(sl.execute)
    # make delayed task
    crop_list = []
    for i in os.listdir(IMAGES_RAW):
        crop_list.append(
            delayed_crop(
                i,
                raster_return_open=False,
                write_file="_cba",
                folder_proc_=CROP_DIR,
            )
        )
    # compute
    cropped_result = dask.compute(crop_list)
    with open(CROPPED_FILES_JSON, "w") as f:
        json.dump(cropped_result[0], f)

2022/04/20 15:13:58 - INFO - Running on Sentinel 2 - l1c data
2022/04/20 15:13:59 - INFO - Processing on /data/sat-images/sentinel2/S2A_MSIL1C_20191010T141741_N0208_R010_T20JLL_20191010T174125.SAFE dir
2022/04/20 15:13:59 - INFO - Processing on /data/sat-images/sentinel2/S2A_MSIL1C_20191109T141741_N0208_R010_T20JLL_20191109T173912.SAFE dir
2022/04/20 15:13:59 - INFO - Using clipping flag: True
2022/04/20 15:13:59 - INFO - Using clipping flag: True
2022/04/20 15:13:59 - INFO - PROCESSING band: B02
2022/04/20 15:13:59 - INFO - PROCESSING band: B02
2022/04/20 15:13:59 - INFO - resample: False - scale factor 1 - true pixel 10.0
2022/04/20 15:13:59 - INFO - Processing on /data/sat-images/sentinel2/S2B_MSIL1C_20171104T142029_N0206_R010_T20JLL_20171104T191711.SAFE dir
2022/04/20 15:13:59 - INFO - Using clipping flag: True
2022/04/20 15:13:59 - INFO - PROCESSING band: B02
2022/04/20 15:13:59 - INFO - Processing on /data/sat-images/sentinel2/S2A_MSIL1C_20200128T141651_N0208_R010_T20JLL_20200128

In [9]:
with open(CROPPED_FILES_JSON) as f:
    cropped_result = json.load(f)

In [10]:
crop_df=pd.DataFrame(cropped_result)

In [11]:
crop_df.head()

,raster,raster_cloud,match,raster_path,raster_cloud_path
0,None,None,TOTAL,../TMP-products-int/images-croped/S2B_MSIL1C_2...,../TMP-products-int/images-croped/S2B_MSIL1C_2...
1,None,None,TOTAL,../TMP-products-int/images-croped/S2B_MSIL1C_2...,../TMP-products-int/images-croped/S2B_MSIL1C_2...
2,None,None,TOTAL,../TMP-products-int/images-croped/S2A_MSIL1C_2...,../TMP-products-int/images-croped/S2A_MSIL1C_2...
3,None,None,TOTAL,../TMP-products-int/images-croped/S2B_MSIL1C_2...,../TMP-products-int/images-croped/S2B_MSIL1C_2...
4,None,None,TOTAL,../TMP-products-int/images-croped/S2A_MSIL1C_2...,../TMP-products-int/images-croped/S2A_MSIL1C_2...


In [12]:
crop_df.to_csv(CROPPED_FILES_CSV)

In [ ]:
# Misc
STATS_BASE = BASE_DIR / "stats_calc"
os.makedirs(STATS_BASE, exist_ok=True)
#
stats_results = []
# func
delayed_raster_stats = delayed(get_raster_stats)
#
for r_dict in cropped_result:
    raster_path = r_dict["raster_path"]
    raster_cloud_path = r_dict["raster_cloud_path"]
    stats_results.append(
        delayed_raster_stats(
            raster_path, raster_cloud_path, gpd_cba_crs, stats_base=STATS_BASE
        )
    )
#
stats_computed = dask.compute(stats_results)

/home/frojo/miniconda3/envs/monitoreo-espacios-verdes/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/frojo/miniconda3/envs/monitoreo-espacios-verdes/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/frojo/miniconda3/envs/monitoreo-espacios-verdes/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/frojo/miniconda3/envs/monitoreo-espacios-verdes/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/frojo/miniconda3/envs/monitoreo-espacios-verdes/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarn

Tenemos el conjunto de resultados para cada raster. Ahora lo aumentamos con
- la cobertura de nubes a nivel de escena
- datos relacionados a espacios verdes (identificadores,etc)


In [17]:
gpd_box = gpd.read_file("./data/cordoba.geojson").to_crs(32720)

In [19]:
stats_clouds = []
stats_file = []
for r_dict in cropped_result:
    raster_path = r_dict["raster_path"]
    raster_cloud_path = r_dict["raster_cloud_path"]
    stats_clouds.append(delayed(get_stats)(gpd_box, raster_cloud_path))
    base_raster = os.path.basename(raster_path)
    stats_file.append(base_raster)

In [20]:
clouds_on_shape = dask.compute(stats_clouds)

In [39]:
df_clouds = pd.DataFrame({"clouds_on_shape": clouds_on_shape[0], "raster": stats_file})
df_clouds.to_csv("./products-int/clouds-on-raster-shape.csv", index=False)

Del geopackage que contiene los poligonos de cada espacio verde solo nos vamos a llevar el osm_id (y name), para poder efectuar los merge con alguna clave.
Entonces para cada csv con los estadisticos calculados:
- leemos el csv
- incluimos las nubes (clouds-on-raster-shape.csv)
- incluimos osm_id y name (espacios-verdes-cba.gpkg)
- calculamos la fecha y hacemos el sort correspondiente.

In [86]:
gpd_cba_crs

,osm_id,fclass,name,cluster_id,area_m2,geometry
0,23300957,park,Plaza de las Américas,78.0,9134.445536,"POLYGON ((386455.904 6521794.986, 386458.932 6..."
1,25622122,park,Plaza San Martín,94.0,14423.739124,"POLYGON ((387461.102 6523682.056, 387575.314 6..."
2,25925819,park,Plaza de la Intendencia,96.0,16577.269919,"POLYGON ((386734.941 6523632.038, 386822.036 6..."
3,25925923,park,Plaza Italia,97.0,5079.075863,"POLYGON ((386824.710 6523756.261, 386897.363 6..."
4,25926534,park,Paseo Sobremonte,98.0,15105.827181,"POLYGON ((386646.376 6523808.037, 386765.951 6..."
...,...,...,...,...,...,...
752,900774390,park,None,3907.0,5246.219251,"POLYGON ((379855.683 6535374.299, 380005.996 6..."
753,901791744,park,None,18291.0,9394.381202,"POLYGON ((378797.793 6536102.539, 378801.807 6..."
754,903836656,park,Plaza Chateau Carreras,18344.0,1215.253994,"POLYGON ((380252.171 6526963.993, 380266.742 6..."
755,924965689,park,None,18731.0,12020.902361,"POLYGON ((381315.897 6516635.545, 381340.863 6..."


In [87]:
aux_df = []
for csv in glob.glob("products-int/stats_calc/*.csv"):
    df_tmp = pd.read_csv(csv, index_col=0)
    df_tmp["clouds_on_raster"] = df_clouds.loc[
        df_clouds.raster == df_tmp["raster"][0], "clouds_on_shape"
    ].values[0]
    df_tmp["osm_id"] = gpd_cba_crs["osm_id"]
    df_tmp["name"] = gpd_cba_crs["name"]
    df_tmp["area_m2"] = gpd_cba_crs["area_m2"]
    df_tmp["date"] = pd.to_datetime(
        df_tmp["raster"].apply(lambda x: x.split("_")[2][:8]), format="%Y%m%d"
    )
    aux_df.append(df_tmp)

In [88]:
df_cat = pd.concat(aux_df)

In [89]:
df_cat.sort_values("date", inplace=True)
df_cat.reset_index(drop=True, inplace=True)
df_cat.to_csv("products-int/espacios-verdes-indexs-cba-2017-2021.csv")

/data/proyectos/espacios-verdes-monitoreo/.pyenv/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(
/data/proyectos/espacios-verdes-monitoreo/.pyenv/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(
/data/proyectos/espacios-verdes-monitoreo/.pyenv/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(
/data/proyectos/espacios-verdes-monitoreo/.pyenv/lib/python3.8/site-packages/geopandas/_compat